In [2]:
import tensorflow as tf

# Create the graph
x = tf.Variable(3, name="x")
y = tf.Variable(4, name="y")
f = x*x*y + y + 2

# TensorFlow session takes care of placing the operations onto devices such as CPU and GPU
# and running them, and it holds all the variable values

# Create a session
sess = tf.Session()
sess.run(x.initializer)
sess.run(y.initializer)
result = sess.run(f)
print(result)
sess.close()



42


In [4]:
# Create a session with with keyword. Session will be auto closed afterwords
with tf.Session() as sess:
    x.initializer.run()
    y.initializer.run()
    result = f.eval()
    print(result)

42


In [6]:
# Don't have to run initialization for each variables. 
# Doesn't actaully perform initialization at this point
init = tf.global_variables_initializer()
with tf.Session() as sess:
    init.run() # actually initializes variables at this point
    result = f.eval()
    print(result)

42


In [7]:
# Any node created is automatically added to the default graph
x1 = tf.Variable(1)
x1.graph is tf.get_default_graph()

True

In [19]:
# Example of managing multiple graphs
# create a new graph and temporarily make it the default graph
graph = tf.Graph()
with graph.as_default():
    x2 = tf.Variable(2)
print(x2.graph is graph)
print(x2.graph is tf.get_default_graph())


True
False


In [21]:
# reset default graph
tf.reset_default_graph()

In [22]:
# TensorFlow automatically determines the set of nodes that it depends on
# and it evaluates these nodes first
w = tf.constant(3)
x = w + 2
y = x + 5
z = x * 3

# w and x gets evaluated twice
with tf.Session() as sess:
    print(y.eval()) #10
    print(z.eval()) #15

10
15


In [23]:
# evaluate y and z in one graph
with tf.Session() as sess:
    y_val, z_val = sess.run([y, z])
    print(y_val) # 10
    print(z_val) # 15

10
15


In [28]:
# Use normal Equation to compute theta
import numpy as np
from sklearn.datasets import fetch_california_housing

housing = fetch_california_housing()
m, n = housing.data.shape
# add constant term
# https://docs.scipy.org/doc/numpy/reference/generated/numpy.c_.html
housing_data_plus_bias = np.c_[np.ones((m,1)), housing.data]

X = tf.constant(housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")

# X transpose
XT = tf.transpose(X)
# Normal Equation
theta = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(XT, X)), XT), y)

with tf.Session() as sess:
    theta_value = theta.eval()
    print(theta_value)
    

[[ -3.74651413e+01]
 [  4.35734153e-01]
 [  9.33829229e-03]
 [ -1.06622010e-01]
 [  6.44106984e-01]
 [ -4.25131839e-06]
 [ -3.77322501e-03]
 [ -4.26648885e-01]
 [ -4.40514028e-01]]


In [66]:
# Use batch gradient descent to compute theta
from sklearn.preprocessing import StandardScaler

# hyper parameter
n_epochs = 1000
learning_rate = 0.01

# normalize data
std_scale = StandardScaler().fit(housing_data_plus_bias)
scaled_housing_data_plus_bias = std_scale.transform(housing_data_plus_bias)

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")

# get the number of the records
m = housing_data_plus_bias.shape[0]

# hand compute the gradient of mse
#gradients = 2/m * tf.matmul(tf.transpose(X), error)

# use tensorflow to compute gradient
# gradients = tf.gradients(mse, [theta])[0]
# training_op = tf.assign(theta, theta - learning_rate * gradients)

# even better way out of box gradient descent!
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch", epoch, "MSE=", mse.eval())
        sess.run(training_op)
        best_theta = theta.eval()

print(best_theta)        

Epoch 0 MSE= 7.97361
Epoch 100 MSE= 4.88693
Epoch 200 MSE= 4.82699
Epoch 300 MSE= 4.81807
Epoch 400 MSE= 4.81379
Epoch 500 MSE= 4.81084
Epoch 600 MSE= 4.80872
Epoch 700 MSE= 4.8072
Epoch 800 MSE= 4.8061
Epoch 900 MSE= 4.80531
[[  8.57028246e-01]
 [  8.09429586e-01]
 [  1.30137146e-01]
 [ -1.99005410e-01]
 [  2.38208801e-01]
 [ -9.98114774e-05]
 [ -3.98587249e-02]
 [ -8.35838497e-01]
 [ -8.02771211e-01]]


In [82]:
# demonstrate saver
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0), name="theta")

init = tf.global_variables_initializer()
saver = tf.train.Saver()

with tf.Session() as sess:
    sess.run(init)
    #example of restore
    #saver.restore(sess, "tmp/my_model_final.ckpt")
    
    for epoch in range(n_epochs):
        # checkpoint every 100 epochs
        if epoch % 100 == 0:
            save_path = saver.save(sess, "tmp/my_model.ckpt")
        sess.run(training_op)
        best_theta = theta.eval()
    save_path = saver.save(sess, "tmp/my_model_final.ckpt")